<a href="https://colab.research.google.com/github/farhadkamangar/CSE5368/blob/master/tensorflow_multi_layer_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from datetime import datetime

%load_ext tensorboard

In [3]:
class DenseLayer(tf.Module):
  def __init__(self, input_dimension, number_of_nodes, name=None):
    super().__init__(name=name)
    self.w = tf.Variable(
      tf.random.normal([input_dimension, number_of_nodes]), name='weights')
    self.b = tf.Variable(tf.zeros([number_of_nodes]), name='biases')
  def calculate_output(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

In [16]:
class MultiLayer_NN(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.layer_1 = DenseLayer(input_dimension=3, number_of_nodes=3)
    self.layer_2 = DenseLayer(input_dimension=3, number_of_nodes=6)

  def __call__(self, x):
    x = self.layer_1.calculate_output(x)
    return self.layer_2.calculate_output(x)

# You have made a model!
my_model = MultiLayer_NN(name="the_model")

# Call it, with random results
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))
print("Weights ", my_model.layer_2.w)
print("Submodules:", my_model.submodules)
for var in my_model.variables:
  print(var, "\n")

Model results: tf.Tensor([[ 3.3240354  0.         5.936384   0.         4.940175  11.829053 ]], shape=(1, 6), dtype=float32)
Weights  <tf.Variable 'weights:0' shape=(3, 6) dtype=float32, numpy=
array([[ 0.8721237 , -0.10960671,  0.96784043, -0.12975301,  1.4159669 ,
        -0.62478286],
       [-0.27331188, -0.7469185 ,  1.3878955 ,  0.7425421 ,  0.13886489,
        -0.63346016],
       [ 0.35855648, -0.23694906,  0.8342599 , -1.3569158 ,  0.49348417,
         2.5020318 ]], dtype=float32)>
Submodules: (<__main__.DenseLayer object at 0x7f1eed16c990>, <__main__.DenseLayer object at 0x7f1eed16c710>)
<tf.Variable 'biases:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'weights:0' shape=(3, 3) dtype=float32, numpy=
array([[-1.1952347 ,  1.1689026 ,  1.3443784 ],
       [ 1.5287184 , -0.7772326 , -0.01542287],
       [ 0.51341724, -1.0981852 ,  1.2464136 ]], dtype=float32)> 

<tf.Variable 'biases:0' shape=(6,) dtype=float32, numpy=array([0., 0., 0., 0.,

In [18]:
class FlexibleDenseLayer(tf.Module):
  def __init__(self, number_of_nodes, name=None):
    super().__init__(name=name)
    self.first_time=True
    self.number_of_nodes=number_of_nodes
    
  def calculate_output(self, x):
    if self.first_time:
      self.w = tf.Variable(tf.random.normal([x.shape[-1], self.number_of_nodes]), name='weights')
      self.b = tf.Variable(tf.zeros([self.number_of_nodes]), name='biases')
      self.first_time=False
    y = tf.matmul(x, self.w) + self.b
    # return tf.nn.hard_tanh(y)
    # return tf.nn.relu(y)
    return y

In [29]:
class MultiLayer_NN(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.layer_1 = FlexibleDenseLayer(number_of_nodes=3)
    self.layer_2 = FlexibleDenseLayer(number_of_nodes=5)

  def __call__(self, x):
    x = self.layer_1.calculate_output(x)
    return self.layer_2.calculate_output(x)

# You have made a model!
my_model = MultiLayer_NN(name="the_model")

# Call it, with random results
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))
print("Weights ", my_model.layer_2.w)
print("Submodules:", my_model.submodules)
for var in my_model.variables:
  print(var, "\n")

Model results: tf.Tensor([[-1.8583658  4.2238026  1.9643538 -0.4884634  4.536718 ]], shape=(1, 5), dtype=float32)
Weights  <tf.Variable 'weights:0' shape=(3, 5) dtype=float32, numpy=
array([[-0.1492263 ,  0.9969619 , -0.46697536, -0.05895721, -0.4458336 ],
       [-0.48961434, -0.8972078 , -1.9212656 , -0.6093035 , -1.3387625 ],
       [ 1.8825271 , -2.0720181 ,  1.570494  ,  1.1322998 , -0.79636824]],
      dtype=float32)>
Submodules: (<__main__.FlexibleDenseLayer object at 0x7f1eecfd2750>, <__main__.FlexibleDenseLayer object at 0x7f1eecfd2250>)
<tf.Variable 'biases:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'weights:0' shape=(3, 3) dtype=float32, numpy=
array([[-0.12552713, -0.13028823, -1.7574427 ],
       [-0.52977663,  0.8169243 ,  0.9981878 ],
       [ 0.15518194, -1.7350601 , -0.04664988]], dtype=float32)> 

<tf.Variable 'biases:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)> 

<tf.Variable 'weights:0' sha

In [30]:
chkp_path = "my_checkpoint"
checkpoint = tf.train.Checkpoint(model=my_model)
checkpoint.write(chkp_path)

'my_checkpoint'

In [31]:
ls my_checkpoint*

my_checkpoint.data-00000-of-00001  my_checkpoint.index


In [32]:
tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/layer_1/b/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('model/layer_1/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 3]),
 ('model/layer_2/b/.ATTRIBUTES/VARIABLE_VALUE', [5]),
 ('model/layer_2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 5])]

In [34]:
new_model = MultiLayer_NN()
new_checkpoint = tf.train.Checkpoint(model=new_model)
new_checkpoint.restore("my_checkpoint")

for var in my_model.variables:
  print(var, "\n")

<tf.Variable 'biases:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'weights:0' shape=(3, 3) dtype=float32, numpy=
array([[-0.12552713, -0.13028823, -1.7574427 ],
       [-0.52977663,  0.8169243 ,  0.9981878 ],
       [ 0.15518194, -1.7350601 , -0.04664988]], dtype=float32)> 

<tf.Variable 'biases:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)> 

<tf.Variable 'weights:0' shape=(3, 5) dtype=float32, numpy=
array([[-0.1492263 ,  0.9969619 , -0.46697536, -0.05895721, -0.4458336 ],
       [-0.48961434, -0.8972078 , -1.9212656 , -0.6093035 , -1.3387625 ],
       [ 1.8825271 , -2.0720181 ,  1.570494  ,  1.1322998 , -0.79636824]],
      dtype=float32)> 

